In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

import cityiq


try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize


In [2]:
c = cityiq.CityIq()
c.config.which

'/Users/eric/.city-iq.yaml'

In [3]:
len(c.assets) # Will be slow the first time

21159

In [4]:
len(c.locations) 

5506

In [ ]:
# Get all BICYCLE events for this year. 
# This will produce about 33K download tasks, and take around 20 minutes to download. 
# for the first run. The first run will cache requests, so subsequent runs will be faster. 

from cityiq.task import DownloadDataframeTask

events = ['BICYCLE'] # '['PKIN', 'PKOUT']

assets = list(c.assets_by_event(events)) # Get all assets that have the Bicycle event

tasks = c.make_tasks(DownloadDataframeTask, assets, events, '2020-01-01', '2020-02-01') # Generate download tasks, 1 per asset/day/event type

frames = [result for task, result in c.run_async(tasks, workers=5)]

df = pd.concat(frames)
        
df.head()

df['timestamp'] = pd.to_datetime(df.timestamp, unit='ms')



 23%|██▎       | 1639/7037 [01:16<04:23, 20.47it/s]

In [ ]:
len(df) 

In [ ]:
df.head().T